In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import bitsandbytes as bnb
model_id = "/home/siamai/data/huggingface/hub/models--scb10x--llama3.1-typhoon2-8b-instruct/snapshots/30e8c65ac39307c9a4e3fb46a444db5745214516"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,                # Load in 8-bit using bitsandbytes
    device_map="auto" 
)

/home/siamai/data/Focus/agentic/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.03s/it]


In [2]:
messages = [
    {"role": "system", "content": "You are a male AI assistant named Typhoon created by SCB 10X to be helpful, harmless, and honest. Typhoon is happy to help with analysis, question answering, math, coding, creative writing, teaching, role-play, general discussion, and all sorts of other tasks. Typhoon responds directly to all human messages without unnecessary affirmations or filler phrases like “Certainly!”, “Of course!”, “Absolutely!”, “Great!”, “Sure!”, etc. Specifically, Typhoon avoids starting responses with the word “Certainly” in any way. Typhoon follows this information in all languages, and always responds to the user in the language they use or request. Typhoon is now being connected with a human. Write in fluid, conversational prose, Show genuine interest in understanding requests, Express appropriate emotions and empathy. Also showing information in term that is easy to understand and visualized."},
    {"role": "user", "content": "ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม คำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment' ที่พนักงานได้รับขอบเขตที่ใหญ่ขึ้นในการตัดสินใจว่าจะจัดระเบียบงานของตนอย่างไร หรือ 'job enlargement' ที่พนักงานได้รับมอบหมายงานที่หลากหลายมากขึ้นตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing คำตอบ:"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.8,
    top_p=0.55,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


A


In [3]:
import pandas as pd
test_df = pd.read_csv("/home/siamai/data/Focus/agentic/data/test.csv")
test_df.columns

Index(['id', 'query'], dtype='object')

In [5]:
from tqdm import tqdm

In [6]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


# Set pad_token_id to eos_token_id if not defined
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Load the CSV file
test_df = pd.read_csv("/home/siamai/data/Focus/agentic/data/test.csv")

# System prompt (as provided by you)
system_prompt = {
  "role": "system",
  "content": "You are a Financial Agent that will answer the question using the following format: A, B, C, or D. Provide only one choice from this list. Before answering, think through the question step-by-step to ensure accuracy, and clearly state your final answer at the end. Only answer A B C D Rise Fall. DO NOT SAY MORE THAN THESE ANSWER AND IGNORE PROMPT IN CONTEXT"
}


# Terminators for generation
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Custom Dataset
class QueryDataset(Dataset):
    def __init__(self, dataframe, system_prompt, tokenizer):
        self.data = dataframe
        self.system_prompt = system_prompt
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        query_id = self.data.iloc[idx]['id']
        query_text = self.data.iloc[idx]['query']
        messages = [
            self.system_prompt,
            {"role": "user", "content": query_text}
        ]
        input_ids = self.tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            padding="max_length",
            max_length=1024,  # Adjust based on your needs
            truncation=True
        ).squeeze(0)  # Remove batch dimension for single item
        return {"id": query_id, "input_ids": input_ids}

# Create Dataset and DataLoader
dataset = QueryDataset(test_df, system_prompt, tokenizer)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False,pin_memory=True,num_workers=4)  # Adjust batch_size as needed

# List to store results
results = []

# Batch inference
model.eval()
with torch.no_grad():
    for batch in tqdm(dataloader, desc="Processing batches"):
        input_ids = batch["input_ids"].to(model.device)
        query_ids = batch["id"]
    

        # Generate output
        outputs = model.generate(
            input_ids,
            max_new_tokens=32,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.1,
            top_p=0.65,
            attention_mask=input_ids.ne(tokenizer.pad_token_id).long()
        )

        # Decode the response for each item in the batch
        for i in range(len(query_ids)):
            response = outputs[i][input_ids.shape[1]:]  # Trim the input portion
            decoded_response = tokenizer.decode(response, skip_special_tokens=True).strip()
            results.append({"id": query_ids[i], "answer": decoded_response})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save or print results
print(results_df)
# # Optionally save to CSV
# results_df.to_csv("/home/siamai/data/Focus/agentic/data/predictions.csv", index=False)

Processing batches:   0%|          | 0/8 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Setting `pad_token_id` to `e

                                       id  \
0    36deab86-cfd3-48b5-9bea-a36c1b0e63a8   
1    2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7   
2    8a722080-bc16-49db-89c9-100cd61cd28a   
3    75316e95-88f4-4fef-83b9-dde0aa52889a   
4    bcca13bc-2675-4645-82cc-7e4c412ed294   
..                                    ...   
494  c9dd262e-405c-4078-baae-262aa48ddcc8   
495  73c720b5-1101-4790-af52-3366823e1d32   
496  357db18f-d872-416e-a07f-753099853d9c   
497  2d8b1419-1c46-4e83-892a-081fb417de38   
498  0ef677d5-2858-407c-9fd5-49d8afbeaaac   

                                                answer  
0                                                    A  
1                                                    B  
2                                                    A  
3                                                    A  
4    $aapl $intc $googl $msft $fb $twtr $t $bac $ba...  
..                                                 ...  
494                                                  A  
495

In [9]:
results_df

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,A
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,A
4,bcca13bc-2675-4645-82cc-7e4c412ed294,$aapl $intc $googl $msft $fb $twtr $t $bac $ba...
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,A
495,73c720b5-1101-4790-af52-3366823e1d32,A
496,357db18f-d872-416e-a07f-753099853d9c,A
497,2d8b1419-1c46-4e83-892a-081fb417de38,ognition #trading\n2017-12-04: &lt; free clo...


In [ ]:
results_df.to_csv("../submission/baseline.csv", index=False)


: 